In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from rdkit.Chem import MolFromSmiles, Lipinski # NumHDonors, NumHAcceptors

In [2]:
df = pd.read_csv("zinc15_train_full.csv")
df = df.loc[:,('smiles', 'mwt', 'logp', 'length', 'cnt_ring', 'ba_bcl2', 'ba_bclxl', 'ba_bclw')]

In [3]:
idx = (df.loc[:,'ba_bcl2'] > 9.069) & (df.loc[:,'ba_bclxl'] > 9.283) & (df.loc[:,'ba_bclw'] > 6.999)
df = df[idx].reset_index(drop=True)

In [4]:
df

,smiles,mwt,logp,length,cnt_ring,ba_bcl2,ba_bclxl,ba_bclw
0,CCCOC(=O)c1cccc(NC(=O)CSc2nnc(C(C)NC(=O)c3cccc...,509.632,4.395,57,3,9.593801,9.423721,7.881210
1,CCCc1cc(C(=O)N2CC(c3ccccc3OC)C3(C2)CN(C(C)C)C(...,519.621,4.839,69,5,9.078078,9.775959,7.851386
2,O=C(O)c1ccc(NC(=O)c2ccc(N3C(=O)C4C5C=CC(C5)C4C...,682.689,4.512,100,9,9.157572,9.435760,7.986024
3,CC(C)(Cc1ccc(Oc2ccc(C(N)=O)cn2)cc1)NCC(O)COc1c...,524.621,4.958,65,5,9.627657,9.306619,7.323000
4,CCNC(=O)C1OC(n2cnc3c(NCC(c4ccccc4)c4ccccc4)nc(...,777.887,1.919,92,7,9.348688,9.299333,7.533735
5,COCCCC1(C(=O)NCCc2cscn2)CNC(=O)c2c3ccccc3nn2CC...,604.733,2.746,66,5,9.725193,9.323896,7.745878
6,N=C(N)NCCCC1NC(=O)C(Cc2ccc(O)cc2)NC(=O)CNC(=O)...,729.843,-1.074,88,4,9.884827,9.707996,8.506519
7,CC(C)(C)c1cc(NC(=O)NCc2ccccc2Sc2ccc3nnc(-c4ccc...,700.290,7.796,80,6,10.100492,9.368319,7.825716
8,CCCc1cc(C(=O)N2CCC(N3CCCC(C)(C)COc4ccccc4Oc4nc...,532.685,6.120,60,5,9.139963,9.368947,7.766162
9,CCOc1ccccc1N1CC(C(=O)Oc2cccc(N3C(=O)C4C5C=CC(C...,512.562,3.601,67,7,9.277481,9.471024,7.866191


In [5]:
for i, smi in tqdm.tqdm(enumerate(df.loc[:,'smiles'].values), total=len(df)):
    
    mol = MolFromSmiles(smi)
    
    df.loc[i,'donor'] = Lipinski.NumHDonors(mol)
    df.loc[i,'acceptor'] = Lipinski.NumHAcceptors(mol)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1349.16it/s]


In [6]:
for i in tqdm.trange(len(df)):
    
    df.loc[i,'violation_Ro5'] = 0
    
    df.loc[i,'violation_Ro5'] += 0 if df.loc[i,'donor'] < 5 else 1
    df.loc[i,'violation_Ro5'] += 0 if df.loc[i,'acceptor'] < 10 else 1
    df.loc[i,'violation_Ro5'] += 0 if df.loc[i,'mwt'] < 500 else 1
    df.loc[i,'violation_Ro5'] += 1 if df.loc[i,'logp'] > 5 else 0

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1235.28it/s]


In [7]:
df.loc[:,'violation_Ro5'].value_counts() / len(df)

1.0    0.500000
2.0    0.333333
3.0    0.166667
Name: violation_Ro5, dtype: float64